<a href="https://colab.research.google.com/github/gbend22/ml-homework/blob/main/experiments/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'ML_Final_Project/walmart-recruiting-store-sales-forecasting'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/ML_Final_Project/walmart-recruiting-store-sales-forecasting


In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')
features = pd.read_csv('features.csv')
df = df.merge(stores, how='left', left_on='Store', right_on='Store')
df = df.merge(features, how='left', left_on=['Store','Date'], right_on=['Store','Date'])
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df,test_size=0.2, random_state=42)

y_train = X_train['Weekly_Sales']
y_test = X_test['Weekly_Sales']
X_train.drop(columns='Weekly_Sales',inplace=True)
X_test.drop(columns='Weekly_Sales',inplace=True)
X_train.head()

,Store,Dept,Date,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
138466,15,3,2011-04-01,False,B,123737,30.34,3.811,NaN,NaN,NaN,NaN,NaN,134.068258,7.658,False
289214,30,25,2010-06-11,False,C,42988,83.51,2.668,NaN,NaN,NaN,NaN,NaN,211.112002,8.200,False
52351,6,27,2010-06-04,False,A,202505,79.44,2.705,NaN,NaN,NaN,NaN,NaN,212.698244,7.092,False
203504,21,49,2011-12-02,False,B,140167,48.72,3.172,3389.10,43.0,1258.32,325.35,8623.67,218.359032,7.441,False
233606,24,55,2012-01-06,False,A,203819,32.86,3.585,7325.68,25367.9,203.51,1745.20,3261.35,136.698129,8.659,False


In [6]:
from sklearn.preprocessing import LabelEncoder

drop_cols = ['Date']
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

categorical_cols = ['Type', 'IsHoliday_y']

for col in categorical_cols:
    if X_train[col].dtype == 'object' or str(X_train[col].dtype).startswith('category') or X_train[col].nunique() < 50:
        encoder = LabelEncoder()
        all_data = pd.concat([X_train[col], X_test[col]], axis=0)
        encoder.fit(all_data)
        X_train[col] = encoder.transform(X_train[col])
        X_test[col] = encoder.transform(X_test[col])

In [7]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [8]:
def weighted_mae(y_true, y_pred, is_holiday):
    weights = is_holiday.apply(lambda x: 5 if x else 1)
    error = np.abs(y_true - y_pred)
    return np.sum(weights * error) / np.sum(weights)

In [9]:
y_pred = model.predict(X_test)
wmae = weighted_mae(y_test, y_pred, X_test['IsHoliday_y'])
print(f"Baseline WMAE: {wmae:.2f}")

Baseline WMAE: 2140.20


In [10]:
import pandas as pd

test = pd.read_csv("test.csv")
stores = pd.read_csv("stores.csv")
features = pd.read_csv("features.csv")

test = test.merge(stores, how='left', on='Store')
test = test.merge(features, how='left', on=['Store', 'Date'])

test = test.drop(columns=['Date'])

test = test.fillna(0)

In [12]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Type', 'IsHoliday_y']

for col in categorical_cols:
    X_train[col] = X_train[col].astype(str)
    test[col] = test[col].astype(str)

    encoder = LabelEncoder()
    all_data = pd.concat([X_train[col], test[col]], axis=0)
    encoder.fit(all_data)

    X_train[col] = encoder.transform(X_train[col])
    test[col] = encoder.transform(test[col])

In [13]:
y_test_pred = model.predict(test)

In [14]:
test_raw = pd.read_csv("test.csv")

submission = pd.DataFrame({
    "Id": test_raw["Store"].astype(str) + "_" +
          test_raw["Dept"].astype(str) + "_" +
          test_raw["Date"].astype(str),
    "Weekly_Sales": y_test_pred
})

submission.to_csv("submission.csv", index=False)